In [1]:
import os
import pandas as pd
from paths import *
from misc_tools import PathUtils as PU
from misc_tools import AudioCut

In [2]:
def assign_syllable_id(group):
    # Initialize syllable_id
    syllable_id = 0
    syllable_ids = []
    
    # Iterate over the rows in the group
    for index, row in group.iterrows():
        if row['syllable_in_id'] == 1:
            # Increment syllable_id when syllable_in_id is 1
            syllable_id += 1
        syllable_ids.append(syllable_id)
    
    # Add the syllable_ids to the group
    group['syllable_id'] = syllable_ids
    return group

In [3]:
for guide_name in ["guide_validation"]: # "guide_train_original", "guide_validation_original.csv", "guide_test_original.csv"
    guide_ = os.path.join(src_eng_, f"{guide_name}_original.csv")
    guide_df = pd.read_csv(guide_)
    guide_df = guide_df.groupby('file').apply(assign_syllable_id).reset_index(drop=True)
    guide_df.to_csv(os.path.join(src_eng_, f"{guide_name}.csv"), index=False)

# Append Syllable UID and Path

In [4]:
for x_name in ["guide_validation.csv"]: # "guide_train", "guide_validation.csv", "guide_test.csv"
    x_ = os.path.join(src_eng_, x_name)

    guide_x = pd.read_csv(x_)

    guide_x["syllable_path"] = guide_x.apply(AudioCut.syllablerecord2filepath, axis=1)

    guide_x["suid"] = guide_x.apply(AudioCut.syllablerecord2suid, axis=1)

    guide_x.to_csv(x_, index=False)

In [2]:
guide_ = os.path.join(src_eng_, "guide_test_complete.csv")
guide_df = pd.read_csv(guide_)

# Add stress mark

In [21]:
import re

In [ ]:
for guide_name in ["guide_validation"]: # "guide_train_original", "guide_validation_original.csv", "guide_test_original.csv"
    guide_ = os.path.join(src_eng_, f"{guide_name}.csv")
    guide_df = pd.read_csv(guide_)
    guide_df = guide_df.groupby('file').apply(assign_syllable_id).reset_index(drop=True)
    guide_df.to_csv(os.path.join(src_eng_, f"{guide_name}.csv"), index=False)

# Append nSample

In [5]:
import torchaudio

In [6]:
def get_nSample(path): 
    waveform, sample_rate = torchaudio.load(path)
    return waveform.size(-1)

In [10]:
guide_ = os.path.join(src_eng_, "guide_test.csv")
guide_df = pd.read_csv(guide_)

In [14]:
recpath = guide_df.iloc[0]["syllable_path"]

In [15]:
get_nSample(os.path.join(train_cut_syllable_, recpath))

10720

# Add stress mark

In [48]:
def getSyllableStressType(row): 
    syllable = row["syllable"]
    stresses = re.findall(r'\d+', syllable)
    if len(stresses) != 1: 
        return "SNA"
    else: 
        return stresses[0]

In [49]:
for guide_name in ["guide_train", "guide_validation", "guide_test"]: # "guide_train_original", "guide_validation_original.csv", "guide_test_original.csv"
    guide_ = os.path.join(src_eng_, f"{guide_name}.csv")
    guide_df = pd.read_csv(guide_)
    small_df = guide_df[["syllable", "suid"]].groupby('suid').first().reset_index()
    small_df["syllable_stress_type"] = small_df.apply(getSyllableStressType, axis=1)
    large_guide_df = pd.merge(guide_df, small_df, how="left", on=["suid", "syllable"])
    large_guide_df.to_csv(os.path.join(src_eng_, f"{guide_name}_complete.csv"), index=False)

: 